In [1]:
from PIL import Image, ImageDraw 
from IPython.display import Image as DImage
import sys, os, hashlib

In [2]:
#INTERFACE
pix_mode    = 1 #compare per-pixel
hash_mode   = 1 #compare hashes
string_mode = 1 #compare hashes of rows and columns
black_mode  = 0 #compare only black pixels
bin_ratio   = 200 #binarization threshold (0-255*3)
basewidth   = 8 #image width after compression (px)

#init directories
if not os.path.exists(os.getcwd()+'/examples/'):
    os.mkdir('examples')
if not os.path.exists(os.getcwd()+'/binary/'):
    os.mkdir('binary')
if not os.path.exists(os.getcwd()+'/resized/'):
    os.mkdir('resized')
examples_path = os.getcwd()+'/examples/'
binary_path = os.getcwd()+'/binary/'
resized_path = os.getcwd()+'/resized/'

#global variables
names = []
values = []
distances = []
strings = []
results = []

RESULT = ''

In [3]:
input_address = input('Input image address:') #input must be in "" ("test.jpg")

Input image address:"test.jpg"


In [4]:
def address_helper(input_img_addr, mode):
    if mode == 'png':
        input_img_addr = input_img_addr.replace('.jpg','.png')
        input_img_addr = input_img_addr.replace('.gif','.png')
    if mode == 'bin':
        input_img_addr = input_img_addr.replace('/examples/','/binary/')
        input_img_addr = input_img_addr.replace('/resized/','/binary/')
    if mode == 'res':
        input_img_addr = input_img_addr.replace('/examples/','/resized/')
        input_img_addr = input_img_addr.replace('/binary/','/resized/')
    if mode == 'cln':
        input_img_addr = input_img_addr.replace('.jpg','')
        input_img_addr = input_img_addr.replace('.gif','')
        input_img_addr = input_img_addr.replace('.png','')
        input_img_addr = input_img_addr.replace(os.getcwd()+'/binary/','')
        input_img_addr = input_img_addr.replace(os.getcwd()+'/resized/','')
    return input_img_addr

In [5]:
def binarization(input_img_addr):
    input_img = Image.open(input_img_addr) 
    draw = ImageDraw.Draw(input_img)
    input_width = input_img.size[0]
    input_height = input_img.size[1]
    input_pixels = input_img.load()
    
    if ".gif" not in input_img_addr:
        for i in range(input_width):
            for j in range(input_height):
                r = input_pixels[i, j][0]
                g = input_pixels[i, j][1]
                b = input_pixels[i, j][2]
                s = r + g + b
                if (s > bin_ratio):
                    r, g, b = 255, 255, 255
                else:
                    r, g, b = 0, 0, 0
                draw.point((i, j), (r, g, b))
    
    input_img_addr = address_helper(input_img_addr,'png')
    input_img.save(address_helper(input_img_addr,'bin'), "PNG")
    del draw

In [6]:
def gethash(input_img_addr):
    input_img = Image.open(input_img_addr)
    return (hashlib.md5(input_img.tobytes()).hexdigest())

In [7]:
def hamdist(str1, str2, prevMin=None):
    diffs = 0
    if len(str1) != len(str2):
        return max(len(str1),len(str2))
    for ch1, ch2 in zip(str1, str2):
        if ch1 != ch2:
            diffs += 1
            if prevMin is not None and diffs > prevMin:
                return None
    return diffs

In [8]:
def resize(input_img_addr, basewidth):
    img = Image.open(input_img_addr)
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth,hsize))
    img.save(address_helper(input_img_addr,'res'), "PNG")

In [9]:
def pix_compare(input_img_addr, example_addr):
    input_img = Image.open(input_img_addr)
    input_pixels = input_img.load()
    example = Image.open(example_addr)
    true = 0 
    
    for i in range(0, input_img.size[0]):
        for j in range(0, input_img.size[1]):
            input_pixel = input_img.getpixel((i, j))
            example_pixel = example.getpixel((i, j))
            if input_pixel == example_pixel:                        
                if not (black_mode == 1 and input_pixels[i, j][0] == 0):
                    true+=1  
    return true

In [10]:
def string_compare(input_img_addr, example_addr):
    input_img = Image.open(input_img_addr)
    example = Image.open(example_addr)
    true = 0
    input_pix = ''
    example_pix = ''
    
    for i in range(0, input_img.size[0]):
        for j in range(0, input_img.size[1]):
            input_pix = input_pix + str(input_img.getpixel((i, j)))
            example_pix = example_pix + str(example.getpixel((i, j)))                   
        true+=hamdist(hashlib.md5(input_pix).hexdigest(),hashlib.md5(example_pix).hexdigest())
        input_pix = ''
        example_pix = ''
        
    for j in range(0, input_img.size[1]):
        for i in range(0, input_img.size[0]):
            input_pix = input_pix + str(input_img.getpixel((i, j)))
            example_pix = example_pix + str(example.getpixel((i, j)))                   
        true+=hamdist(hashlib.md5(input_pix).hexdigest(),hashlib.md5(example_pix).hexdigest())
        input_pix = ''
        example_pix = ''
        
    return true

In [11]:
def compare1to1(input_img_addr, example_addr):
    example_addr = address_helper(example_addr,'png')
    input_img_addr = os.getcwd()+'/binary/'+address_helper(input_img_addr,'png')
    
    if pix_mode == 1:      
        values.append(pix_compare(input_img_addr, example_addr))
        
    if string_mode == 1:
        strings.append(string_compare(input_img_addr, example_addr))
        
    if hash_mode == 1:
        resize(input_img_addr, basewidth)
        resize(example_addr, basewidth)
        
        distances.append(hamdist(gethash(address_helper(example_addr,'res')), 
                                 gethash(address_helper(input_img_addr,'res'))))
            
    example_addr = address_helper(example_addr, 'cln')
    names.append(example_addr)   

In [12]:
def compare1toN(input_img_addr):
    #delete previous values
    global names
    global values
    global distances
    global strings
    names = []
    values = []
    distances = []
    strings = []
    
    examples = os.listdir(examples_path)
    for i in range(len(examples)):
        if ".gif" in examples[i] or ".jpg" in examples[i] or ".png" in examples[i]:
            binarization(examples_path+examples[i])
            compare1to1(input_img_addr,binary_path+examples[i])

In [13]:
def print1toN():
    if pix_mode == 1:
        print('Pixel mode:')
        for i in range(len(names)):
            print(names[i],values[i])
    if hash_mode == 1:
        print('Hash mode:')
        for i in range(len(names)):
            print(names[i],distances[i])
    if string_mode == 1:
        print('String mode:')
        for i in range(len(names)):
            print(names[i],strings[i])

In [14]:
def result1toN():
    global results
    results = ['']
    if pix_mode == 1 and values != []:
        results.append(names[values.index(max(values))])
        print('Pixel mode:',names[values.index(max(values))])
    if hash_mode == 1 and distances != []:
        results.append(names[distances.index(min(distances))])
        print('Hash mode:',names[distances.index(min(distances))])
    if hash_mode == 1 and distances != []:
        results.append(names[distances.index(min(strings))])
        print('String mode:',names[distances.index(min(strings))])
    print('Result:',max(set(results), key=results.count))

In [15]:
binarization(input_address)
compare1toN(input_address)
print1toN()
result1toN()

Pixel mode:
('test', 47718)
('ctest', 38706)
('b', 47416)
('compare', 44884)
('a', 47718)
Hash mode:
('test', 0)
('ctest', 32)
('b', 32)
('compare', 27)
('a', 0)
String mode:
('test', 0)
('ctest', 9560)
('b', 1645)
('compare', 13063)
('a', 0)
('Pixel mode:', 'test')
('Hash mode:', 'test')
('String mode:', 'test')
('Result:', 'test')
